# Recursive Least Squares

In [1]:
import numpy as np

class plant(object):
    
    def __init__(self, h):
        '''
        h - numpy array representing impulse response of system
        '''
        self.h = h
        self.data_length = h.shape[0]
        self.data = np.zeros(self.data_length)
        
    def run_plant(self, x):
        '''
        x - scalar representing next value ingested by the plant
        '''
        self.data = np.roll(self.data,1)
        self.data[0] = x
        return np.sum(self.h*self.data)

In [2]:
sys = plant(np.array([1.,2.,3.,4.,5.]))

In [3]:
for x in np.random.randn(10):
    print(sys.run_plant(x))

-0.49782064593871594
0.876962885407775
2.1596920293258943
5.593756471880458
9.81281753349613
17.822144338988288
12.55330220760541
18.194880356864456
10.33530982205146
9.86316730742638


In [4]:
class RLS(object):
    '''
    Recursive least squares estimator
    '''
    
    def __init__(self, N):
        '''
        N is the desired length of the impulse response
        '''
        self.h = np.zeros(N)
        self.x_data = np.zeros(N)
        self.P = 1e6*np.eye(N)
        self.e = 0
        
    def filt(self,x,y):
        '''
        Filters the data based on the 
        input - x
        desired - y
        '''
        self.x_data = np.roll(self.x_data, 1)
        self.x_data[0] = x
        
        k = self.P@self.x_data/(1. + self.x_data@self.P@self.x_data)
        self.e = y - self.x_data@self.h
        self.h += k*self.e
        self.P -= np.outer(k,self.x_data@self.P)

## Estimation using filter of correct length

In [6]:
rls_filt = RLS(5)
sys = plant(np.array([1.,2.,3.,4.,5.]))

for x in np.random.randn(100):
    y = sys.run_plant(x)
    rls_filt.filt(x,y)

print(rls_filt.h)
print(rls_filt.e)

[1.         1.99999999 2.99999997 3.99999996 4.99999995]
-1.6139770764311834e-07


## Estimation using filter too short

In [7]:
rls_filt = RLS(4)
sys = plant(np.array([1.,2.,3.,4.,5.]))

for x in np.random.randn(100):
    y = sys.run_plant(x)
    rls_filt.filt(x,y)

print(rls_filt.h)
print(rls_filt.e)

[0.67984359 2.04549259 3.04227276 3.34375769]
-5.737653496331016


As seen, the error is larger and the filter fails to estimate the impulse response.